In [1]:
# Config Variables

# Define UCI Datasets to pull
# https://archive.ics.uci.edu/ml/datasets.php
uci_datasets = [165]

# Use all data (vs just training data) for OLS to determine co-efficient sign
ols_use_all_data = True

# Training set sizes to use (only training set sizes < 60% of total dataset size will be used)
training_set_sizes = [10, 40, 160, 610, 2560, 10240, 40960]
chain_lengths = [20000, 20000, 100000, 100000, 100000, 100000, 100000]

# Number of etas (prior standard deviation) to test between 1e-04 and 1e05 (recommend 20)
# Limits at 0 and infinity are calculated separately
# Note: In practice, sigma^2 (likelihood variance) is set to 1/eta^2 and
#       lambda (penalty) is set to 1/eta^4 
num_etas = 20

# Number of folds for cross-validation (set to 300 in 2018 paper, recommend testing with 5 to see if it works)
cv_folds = 5

# Overwrite model if it's already saved to file
overwrite_model = False

In [2]:
# Load necessary libraries and helper functions

# import helper functions from helpers/data_manipulation.py
from helpers.data_manipulation import *

# import half_ridge functions from helpers/half_ridge.py
from helpers.half_ridge import *

# import dataset class
from helpers.model import Model

# import os and pickle
import os
import pickle
 
# Pandas and numpy for data manipulation
import pandas as pd
import numpy as np
np.random.seed(42)

# Matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib.ticker import FuncFormatter
import matplotlib
matplotlib.rcParams['font.size'] = 16
matplotlib.rcParams['figure.figsize'] = (9, 9)

import seaborn as sns
import time

from IPython.core.pylabtools import figsize

# Scipy helper functions
from scipy.stats import percentileofscore
from scipy import stats

# Standard ML Models for comparison
from sklearn.linear_model import LinearRegression

# Splitting data into training/testing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error

# Distributions
import scipy

# PyMC3 for Bayesian Inference
import arviz as az
import pymc as pm

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [3]:
# Load the ICU dataset, normalize the data, and split into training, validation, and test sets

# Loop over each dataset, loaded from file and formatted
for dataset in uci_datasets:
    model = Model()
    model.import_from_file('uci_ml_datasets/ucirepo_' + str(dataset) + '.csv')
    model.format_data()
    training_set_values = model.set_up_training_values(training_set_sizes, .6)

    print(training_set_values)
    
    # Initialize the comparison results
    binary_comparisons = {}

    # loop over the training set values
    for training_set_index, training_set_value in enumerate(training_set_values):
        print(f"Training Set Index: {training_set_index}, Training Set Value: {training_set_value}")

        # Check if a file exists for the dataset in the dataset_models folder
        # if it does, skip the rest of the code and continue with the loop
        # if it doesn't, continue with the rest of the code
        if os.path.isfile(f"models/uci_repo_{dataset}/{training_set_value}.csv") and not overwrite_model:
            print(f"Model already exists for dataset {dataset} and training_set_value {training_set_value}. Skipping...")
            continue

        # Generate cross-validation indices
        cv = cv_indexing(cv_folds, model.X.shape[0], training_set_value)

        # Initialize the comparison results for this training set value
        binary_comparisons[training_set_value] = {}

        # Vary eta values from 10^-4 to 10^5 and loop over them
        etas = np.logspace(-4, 5, num=num_etas)
        etas = np.insert(etas, 0, 0)
        etas = np.append(etas, np.inf)
        for eta in etas:

            # Initialize the comparison results for this eta
            binary_comparisons[training_set_value][eta] = []

            # Crossvalidate over the folds for a specific tau
            for i in range(cv_folds):

                # Print info about this iteration
                print(f"Training Set Value: {training_set_value}, Eta: {eta}, Fold: {i} of {range(cv_folds)}")

                # Get training and test indices for the i-th fold
                train_indices = np.where(cv[i] == 1)[0]
                test_indices = np.where(cv[i] == 0)[0]
                
                # Split the data
                X_train, X_test = model.X.iloc[train_indices], model.X.iloc[test_indices]
                y_train, y_test = model.y.iloc[train_indices], model.y.iloc[test_indices]

                # Determine the signs of the coefficients using OLS
                ols_coefficients = model.determine_signs_of_coefficients(ols_use_all_data, X_train, y_train)

                # Run the half ridge regression with mcmc
                #half_ridge_trace = half_ridge(X_train, y_train, model.ols_coefficients, prior_eta=eta)
                #var_dict = {}
                #for variable in half_ridge_trace.posterior:
                #    var_dict[variable] = half_ridge_trace.posterior[variable].values.flatten()
                # Results into a dataframe
                #var_weights = pd.DataFrame(var_dict)
                # Means for all the weights
                #var_means_array = var_weights.mean(axis=0)
                # Convert the dataframe var_means_array to a dictionary with the column names as keys
                #half_ridge_coefficients = {col: var_means_array[col] for col in var_weights.columns}
                
                # Run the half ridge regression with rejection sampling
                start_time = time.time()
                half_ridge_coefficients = half_ridge_rejection_sampling(ols_coefficients, X_train, y_train, eta, chain_lengths[training_set_index])
                end_time = time.time()
                duration = end_time - start_time
                print(f"Half ridge regression rejection sampling took {duration} seconds")

                # Calculate the binary comparisons for the models
                sorted_half_ridge_coefficients = {k: half_ridge_coefficients[k] for k in sorted(half_ridge_coefficients)}
                sorted_half_ridge_coefficients['intercept'] = 0
                print('half-ridge coefficients')
                print(sorted_half_ridge_coefficients)
                binary_comparison = calculate_binary_comparison(sorted_half_ridge_coefficients, X_test, y_test)
                binary_comparisons[training_set_value][eta].append(binary_comparison)
                print('binary comparisons')
                print(binary_comparison)   

    with open('model_results/ucirepo_' + str(dataset) + '.csv', 'wb') as file:
        pickle.dump(binary_comparisons, file)


[10, 40, 160, 610]
Training Set Index: 0, Training Set Value: 10
Training Set Value: 10, Eta: 0.0, Fold: 0 of range(0, 5)
Running half-ridge regression...
{'Cement': 0.7978845608028654, 'Superplasticizer': -0.7978845608028654, 'Age': -0.7978845608028654, 'Water': -0.7978845608028654, 'Fine Aggregate': -0.7978845608028654, 'Coarse Aggregate': -0.7978845608028654, 'Blast Furnace Slag': -0.7978845608028654, 'Fly Ash': -0.7978845608028654}
Half ridge regression rejection sampling took 7.033348083496094e-05 seconds
half-ridge coefficients
{'Age': -0.7978845608028654, 'Blast Furnace Slag': -0.7978845608028654, 'Cement': 0.7978845608028654, 'Coarse Aggregate': -0.7978845608028654, 'Fine Aggregate': -0.7978845608028654, 'Fly Ash': -0.7978845608028654, 'Superplasticizer': -0.7978845608028654, 'Water': -0.7978845608028654, 'intercept': 0}
binary comparisons
0.46
Training Set Value: 10, Eta: 0.0, Fold: 1 of range(0, 5)
Running half-ridge regression...
{'Cement': 0.7978845608028654, 'Superplastici

/workspaces/codespaces-jupyter/helpers/half_ridge.py:149: RuntimeWarning: Mean of empty slice
  post_weights = np.nanmean(samples1, axis=0)


binary comparisons
0.703
Training Set Value: 10, Eta: 1.8329807108324339, Fold: 1 of range(0, 5)
Running half-ridge regression...
Half ridge regression rejection sampling took 0.006690502166748047 seconds
half-ridge coefficients
{'Age': -0.15044696865679263, 'Blast Furnace Slag': -1.9536993971058199, 'Cement': 1.195516786507513, 'Coarse Aggregate': -0.7098744191294153, 'Fine Aggregate': -4.02170702758751, 'Fly Ash': -0.1794007489733467, 'Superplasticizer': -1.1402645486531087, 'Water': -3.0076543621831724, 'intercept': 0}
binary comparisons
0.588
Training Set Value: 10, Eta: 1.8329807108324339, Fold: 2 of range(0, 5)
Running half-ridge regression...
Half ridge regression rejection sampling took 0.00676274299621582 seconds
half-ridge coefficients
{'Age': -0.11756158318110099, 'Blast Furnace Slag': -0.33756943000911277, 'Cement': 0.21575045413455185, 'Coarse Aggregate': -0.8857297678603572, 'Fine Aggregate': -0.48581294051801327, 'Fly Ash': -0.36233078768938065, 'Superplasticizer': -0.10

/workspaces/codespaces-jupyter/helpers/half_ridge.py:149: RuntimeWarning: Mean of empty slice
  post_weights = np.nanmean(samples1, axis=0)


binary comparisons
0.691
Training Set Value: 10, Eta: 5.455594781168514, Fold: 1 of range(0, 5)
Running half-ridge regression...
Half ridge regression rejection sampling took 0.03320646286010742 seconds
half-ridge coefficients
{'Age': 0.6985349117448929, 'Blast Furnace Slag': -0.11900582806200281, 'Cement': 1.5089065782131243, 'Coarse Aggregate': 0.7425795608693303, 'Fine Aggregate': -1.3049627621611073, 'Fly Ash': 0.7601170841734838, 'Superplasticizer': -0.0073350442032962225, 'Water': -0.5059854869612874, 'intercept': 0}


...

binary comparisons
0.622
Training Set Value: 610, Eta: 33598.18286283774, Fold: 2 of range(0, 5)
Running half-ridge regression...
Half ridge regression rejection sampling took 0.04992556571960449 seconds
half-ridge coefficients
{'Age': -892696544.2082748, 'Blast Furnace Slag': -890584959.5319138, 'Cement': 908064947.937328, 'Coarse Aggregate': -955770336.2931805, 'Fine Aggregate': -887764360.6736556, 'Fly Ash': -918120093.6343566, 'Superplasticizer': -876834660.0379034, 'Water': -896574020.3869593, 'intercept': 0}
binary comparisons
0.457
Training Set Value: 610, Eta: 33598.18286283774, Fold: 3 of range(0, 5)
Running half-ridge regression...
Half ridge regression rejection sampling took 0.03947091102600098 seconds
half-ridge coefficients
{'Age': -842048855.3341707, 'Blast Furnace Slag': -851459595.6908666, 'Cement': 821031265.444242, 'Coarse Aggregate': -969839259.6464537, 'Fine Aggregate': -902049433.095334, 'Fly Ash': -873006218.2197667, 'Superplasticizer': -918104972.6201371, 'Water

/workspaces/codespaces-jupyter/helpers/half_ridge.py:149: RuntimeWarning: Mean of empty slice
  post_weights = np.nanmean(samples1, axis=0)


binary comparisons
0.627
Training Set Value: 610, Eta: 100000.0, Fold: 1 of range(0, 5)
Running half-ridge regression...
Half ridge regression rejection sampling took 0.05923008918762207 seconds
half-ridge coefficients
{'Age': 0.40236383535358755, 'Blast Furnace Slag': 0.5159683873451046, 'Cement': 0.7335778193093351, 'Coarse Aggregate': 0.06673465949138079, 'Fine Aggregate': 0.08309483963378568, 'Fly Ash': 0.3178223585997527, 'Superplasticizer': 0.13851589766147285, 'Water': -0.19149892383746014, 'intercept': 0}
binary comparisons
0.649
Training Set Value: 610, Eta: 100000.0, Fold: 2 of range(0, 5)


/workspaces/codespaces-jupyter/helpers/half_ridge.py:149: RuntimeWarning: Mean of empty slice
  post_weights = np.nanmean(samples1, axis=0)


Running half-ridge regression...
Half ridge regression rejection sampling took 0.03505086898803711 seconds
half-ridge coefficients
{'Age': -7868721726.384825, 'Blast Furnace Slag': -8020012997.545368, 'Cement': 7540157462.935725, 'Coarse Aggregate': -8236118333.917766, 'Fine Aggregate': -8074873731.141058, 'Fly Ash': -7844889559.640875, 'Superplasticizer': -8107589466.870805, 'Water': -7812233393.416296, 'intercept': 0}
binary comparisons
0.473
Training Set Value: 610, Eta: 100000.0, Fold: 3 of range(0, 5)
Running half-ridge regression...
Half ridge regression rejection sampling took 0.035455942153930664 seconds
half-ridge coefficients
{'Age': -8254475703.957775, 'Blast Furnace Slag': -8540624210.124124, 'Cement': 7897589603.063478, 'Coarse Aggregate': -8084245841.822736, 'Fine Aggregate': -8151865843.185234, 'Fly Ash': -8241803596.270858, 'Superplasticizer': -8648610542.920622, 'Water': -8079922705.28779, 'intercept': 0}
binary comparisons
0.443
Training Set Value: 610, Eta: 100000.0,